In [4]:
import os
import subprocess
from pydub import AudioSegment

def extract_audio_from_video(video_path, audio_path):
    command = ["wsl", "ffmpeg", "-i", video_path, "-vn", audio_path]
    subprocess.run(command)

track = AudioSegment.from_mp3("./files/video.mp3")

track.duration_seconds

2978.3771428571426